## Recuperador Híbrido - Combinando Recuperadores Densos y Dispersos

Un recuperador híbrido combina dos enfoques complementarios:
- **Recuperación Densa**: Utiliza embeddings vectoriales para búsqueda semántica (captura el significado)
- **Recuperación Dispersa**: Utiliza técnicas de coincidencia de palabras clave como BM25 (captura términos exactos)

Esta combinación mejora la calidad de recuperación al aprovechar las fortalezas de ambos métodos.

In [1]:
# Importación de librerías necesarias para la búsqueda híbrida

# FAISS: Librería de Facebook para búsqueda de similitud vectorial eficiente
from langchain_community.vectorstores import FAISS

# HuggingFaceEmbeddings: Para generar embeddings usando modelos de HuggingFace
from langchain_huggingface import HuggingFaceEmbeddings

# BM25Retriever: Implementación del algoritmo BM25 para recuperación dispersa basada en palabras clave
from langchain_community.retrievers import BM25Retriever

# EnsembleRetriever: Combina múltiples recuperadores con pesos específicos
from langchain.retrievers import EnsembleRetriever

# Document: Clase para representar documentos con contenido y metadatos
from langchain.schema import Document

c:\Udemy\RAGBootcamp\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Paso 1: Crear documentos de ejemplo
# Creamos una lista de documentos con diferentes temáticas para probar el sistema de recuperación
docs = [
    # Documento sobre LangChain y su propósito
    Document(page_content="LangChain ayuda a construir aplicaciones LLM."),
    # Documento sobre bases de datos vectoriales
    Document(page_content="Pinecone es una base de datos vectorial para búsqueda semántica."),
    # Documento con información geográfica
    Document(page_content="La Torre Eiffel está ubicada en París."),
    # Documento sobre aplicaciones de IA agéntica con Langchain
    Document(page_content="Langchain puede ser usado para desarrollar aplicaciones de IA agéntica."),
    # Documento sobre las capacidades de Langchain
    Document(page_content="Langchain tiene muchos tipos de recuperadores.")
]

# Paso 2: Configurar el Recuperador Denso (FAISS + HuggingFace)
# Inicializamos el modelo de embeddings de HuggingFace
# all-MiniLM-L6-v2 es un modelo ligero y eficiente que genera vectores de 384 dimensiones
embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

# Creamos una base de datos vectorial FAISS a partir de los documentos
# FAISS convierte cada documento en un vector usando el modelo de embeddings
dense_vectorstore = FAISS.from_documents(docs, embedding_model)

# Convertimos el almacén vectorial en un recuperador
# Este recuperador realizará búsquedas por similitud semántica
dense_retriever = dense_vectorstore.as_retriever()

In [3]:
# Paso 3: Configurar el Recuperador Disperso (BM25)
# BM25 es un algoritmo de ranking que usa frecuencia de términos y longitud de documentos
# Es excelente para coincidencias exactas de palabras clave
sparse_retriever = BM25Retriever.from_documents(docs)

# Configuramos el número de documentos más relevantes a recuperar (top-k)
# k=3 significa que devolverá los 3 documentos más relevantes
sparse_retriever.k = 3

# Paso 4: Combinar recuperadores con EnsembleRetriever
# El recuperador híbrido combina los resultados de ambos recuperadores
hybrid_retriever = EnsembleRetriever(
    # Lista de recuperadores a combinar: denso (semántico) y disperso (palabras clave)
    retrievers=[dense_retriever, sparse_retriever],
    # Pesos para cada recuperador: 70% denso, 30% disperso
    # Los pesos determinan la importancia relativa de cada método en los resultados finales
    weight=[0.7, 0.3]
)

In [4]:
# Mostrar la configuración del recuperador híbrido
# Esto nos permite verificar cómo está configurado el sistema
hybrid_retriever

EnsembleRetriever(retrievers=[VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000002E773A33980>, search_kwargs={}), BM25Retriever(vectorizer=<rank_bm25.BM25Okapi object at 0x000002E770DCA660>, k=3)], weights=[0.5, 0.5])

In [5]:
# Paso 5: Realizar una consulta y obtener resultados
# Definimos una pregunta que queremos responder usando nuestro sistema de recuperación
query = "¿Cómo puedo construir una aplicación usando LLMs?"

# Invocamos el recuperador híbrido con nuestra consulta
# El sistema combinará los resultados de búsqueda semántica (FAISS) y de palabras clave (BM25)
results = hybrid_retriever.invoke(query)

# Paso 6: Mostrar los resultados recuperados
# Iteramos sobre cada documento recuperado y lo mostramos
for i, doc in enumerate(results):
    # Mostramos el número del documento y su contenido
    print(f"\n🔹 Documento {i+1}:\n{doc.page_content}")


🔹 Documento 1:
LangChain ayuda a construir aplicaciones LLM.

🔹 Documento 2:
Pinecone es una base de datos vectorial para búsqueda semántica.

🔹 Documento 3:
Langchain tiene muchos tipos de recuperadores.

🔹 Documento 4:
Langchain puede ser usado para desarrollar aplicaciones de IA agéntica.


### Pipeline RAG con Recuperador Híbrido

Ahora construiremos un pipeline RAG (Retrieval-Augmented Generation) completo que:
1. Recupera documentos relevantes usando el recuperador híbrido
2. Utiliza esos documentos como contexto para un LLM
3. Genera una respuesta fundamentada en la información recuperada

In [6]:
# Importación de componentes para construir el pipeline RAG

# init_chat_model: Función para inicializar modelos de chat de diferentes proveedores
from langchain.chat_models import init_chat_model

# PromptTemplate: Para crear plantillas de prompts con variables dinámicas
from langchain.prompts import PromptTemplate

# create_stuff_documents_chain: Crea una cadena que "rellena" documentos en un prompt
from langchain.chains.combine_documents import create_stuff_documents_chain

# create_retrieval_chain: Combina recuperación de documentos con generación de respuestas
from langchain.chains.retrieval import create_retrieval_chain

In [7]:
# Paso 5: Crear la plantilla de prompt
# Definimos cómo queremos que el LLM use el contexto recuperado para responder
prompt = PromptTemplate.from_template("""
Responde la pregunta basándote en el contexto a continuación.

Contexto:
{context}

Pregunta: {input}
""")

# Paso 6: Inicializar el modelo de lenguaje (LLM)
# Usamos GPT-3.5-turbo de OpenAI con temperatura 0.2
# temperature=0.2 significa respuestas más determinísticas y menos creativas
llm = init_chat_model("openai:gpt-3.5-turbo", temperature=0.2)

# Mostrar la configuración del LLM
llm

ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x000002E7759D3CB0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000002E775F7F230>, root_client=<openai.OpenAI object at 0x000002E773C7FE60>, root_async_client=<openai.AsyncOpenAI object at 0x000002E775ACA570>, temperature=0.2, model_kwargs={}, openai_api_key=SecretStr('**********'), stream_usage=True)

In [8]:
# Paso 7: Crear la cadena de documentos
# Esta cadena toma los documentos recuperados y los combina con el prompt
# "stuff" significa que todos los documentos se insertan directamente en el prompt
document_chain = create_stuff_documents_chain(llm=llm, prompt=prompt)

# Paso 8: Crear la cadena RAG completa
# Esta cadena combina:
# 1. El recuperador híbrido (para obtener documentos relevantes)
# 2. La cadena de documentos (para generar la respuesta con el LLM)
rag_chain = create_retrieval_chain(retriever=hybrid_retriever, combine_docs_chain=document_chain)

# Mostrar la estructura de la cadena RAG
rag_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | EnsembleRetriever(retrievers=[VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000002E773A33980>, search_kwargs={}), BM25Retriever(vectorizer=<rank_bm25.BM25Okapi object at 0x000002E770DCA660>, k=3)], weights=[0.5, 0.5]), kwargs={}, config={'run_name': 'retrieve_documents'}, config_factories=[])
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
            | PromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, template='\nResponde la pregunta basándote en el contexto a continuación.\n\nContexto:\n{context}\n\nPregunta: {input}\n')
            | Cha

In [9]:
# Paso 9: Realizar una pregunta al sistema RAG
# Creamos un diccionario con nuestra consulta
query = {"input": "¿Cómo puedo construir una aplicación usando LLMs?"}

# Invocamos la cadena RAG completa
# Esto ejecutará: recuperación de documentos → inserción en prompt → generación de respuesta
response = rag_chain.invoke(query)

# Paso 10: Mostrar los resultados
# Imprimimos la respuesta generada por el LLM
print("✅ Respuesta:\n", response["answer"])

# Mostramos los documentos fuente que se usaron para generar la respuesta
print("\n📄 Documentos Fuente:")
for i, doc in enumerate(response["context"]):
    # Para cada documento, mostramos su número y contenido
    print(f"\nDoc {i+1}: {doc.page_content}")

✅ Respuesta:
 Para construir una aplicación utilizando LLMs, puedes utilizar LangChain, ya que esta plataforma ayuda a construir aplicaciones LLM. Además, puedes aprovechar Pinecone como una base de datos vectorial para búsqueda semántica, y utilizar los diferentes tipos de recuperadores que ofrece LangChain. De esta manera, podrás desarrollar aplicaciones de IA agéntica utilizando LLMs.

📄 Documentos Fuente:

Doc 1: LangChain ayuda a construir aplicaciones LLM.

Doc 2: Pinecone es una base de datos vectorial para búsqueda semántica.

Doc 3: Langchain tiene muchos tipos de recuperadores.

Doc 4: Langchain puede ser usado para desarrollar aplicaciones de IA agéntica.
